    000_16_230114_141_subway_transer_01
    
    data : 002_수도권2호선_주소데이터_20221122.csv
     save : subway_info_02.csv
     display : subway_info_02.html

In [16]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [17]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [18]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [19]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/002_수도권2호선_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,서울교통공사,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 396
1,서울교통공사,2호선,강변(동서울터미널),서울특별시 광진구 구의동 546-6 강변역(2호선),서울특별시 광진구 강변역로 53(구의동)
2,서울교통공사,2호선,건대입구,서울특별시 광진구 화양동 7-3 2호선 건대입구역(2호선),서울특별시 광진구 아차산로 243(화양동)
3,서울교통공사,2호선,교대(법원·검찰청),서울특별시 서초구 서초동 1748-4 교대역(2호선),서울특별시 서초구 서초대로 294(서초동)
4,서울교통공사,2호선,구로디지털단지,서울특별시 구로구 구로동 810-3 구로디지털단지역(2호선),서울특별시 구로구 도림천로 477(구로동)


In [20]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  51 non-null     object
 1   선명       51 non-null     object
 2   역명       51 non-null     object
 3   지번주소     51 non-null     object
 4   도로명주소    51 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


In [21]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [22]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [23]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["도로명주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["도로명주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,서울교통공사,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 396,958307.262586959,1944427.77368496
1,서울교통공사,2호선,강변(동서울터미널),서울특별시 광진구 구의동 546-6 강변역(2호선),서울특별시 광진구 강변역로 53(구의동),964189.260209394,1948504.98227089
2,서울교통공사,2호선,건대입구,서울특별시 광진구 화양동 7-3 2호선 건대입구역(2호선),서울특별시 광진구 아차산로 243(화양동),961938.804402788,1949098.10728428
3,서울교통공사,2호선,교대(법원·검찰청),서울특별시 서초구 서초동 1748-4 교대역(2호선),서울특별시 서초구 서초대로 294(서초동),957023.889636057,1943840.53226575
4,서울교통공사,2호선,구로디지털단지,서울특별시 구로구 구로동 810-3 구로디지털단지역(2호선),서울특별시 구로구 도림천로 477(구로동),947058.58393145,1943074.80772245


In [24]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [25]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,서울교통공사,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 396,958307.262586959,1944427.77368496,127.028323,37.498170
1,서울교통공사,2호선,강변(동서울터미널),서울특별시 광진구 구의동 546-6 강변역(2호선),서울특별시 광진구 강변역로 53(구의동),964189.260209394,1948504.98227089,127.094666,37.535167
2,서울교통공사,2호선,건대입구,서울특별시 광진구 화양동 7-3 2호선 건대입구역(2호선),서울특별시 광진구 아차산로 243(화양동),961938.804402788,1949098.10728428,127.069164,37.540422
3,서울교통공사,2호선,교대(법원·검찰청),서울특별시 서초구 서초동 1748-4 교대역(2호선),서울특별시 서초구 서초대로 294(서초동),957023.889636057,1943840.53226575,127.013838,37.492818
4,서울교통공사,2호선,구로디지털단지,서울특별시 구로구 구로동 810-3 구로디지털단지역(2호선),서울특별시 구로구 도림천로 477(구로동),947058.58393145,1943074.80772245,126.901167,37.485399


In [26]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [27]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [28]:
#5 seoul university 
map

In [29]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_02.html")


In [30]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
#subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_02.csv", index = False)
